In [15]:
import requests
import string
import random
import datetime
import os
import json
import re




# Generate unic id
def generate_id(long:int):
    # def characters
    characters = string.ascii_letters + string.digits

    # Generate random id
    generated_id = ''.join(random.choice(characters) for _ in range(long))

    return generated_id
# Save a detailed log
def changeLog(txt:str):
    current_time = datetime.datetime.now()
    log_txt = f"{current_time}: "+f"{txt}"+f'\n'

    # folder name
    log_folder = "./logs"
    # logfile name
    logFile = os.path.join(log_folder, f"log{datetime.date.today()}.txt")
    try:
        with open(logFile,"a") as file:
            file.write(log_txt)
    except:
        # create folder if not exist
        if not os.path.exists(log_folder):
            os.makedirs(logFile)
        # create file if not exist
        if not os.path.exists(logFile):
            open(logFile, 'w').close()

# Get Info about DataSets filtered by name
def getDataSetInfoByName(name):
    """
    :param name: name used to filter in to data sets
    :return: This method return the basic information about specific dataset filtered by id
    """

    try:
        session = login()
        url = f"http://localhost:8080/api/dataSets.json?"
        params = {
            "filter":f"name:like:{name}"
        }
        response = session.get(url, headers={'content-type': 'application/json'}, params=params)
        changeLog(f"[FETCHING DATA-SET INFO] >> {response.text}")
        return response.text
    except Exception:
        changeLog("Unspected exception")
# Get Info about DataSets filtered by name
def getOrgUnitInfoByName(name):
    """
    :param name: name used to filter in to OrgUnits
    :return: This method return the basic information about specific organisation unit filtered by id
    """
    try:
        session = login()
        url = f"http://localhost:8080/api/organisationUnits.json?"
        params = {
            "filter": f"name:like:{name}"
        }
        response = session.get(url, headers={'content-type': 'application/json'}, params=params)
        changeLog(f"[FETCHING ORG-UNIT INFO] >> {response.text}")
        return response.text
    except Exception:
        changeLog("Unspected exception")
# Get Info about DataElements filtered by name
def getDataElementInfoByName(name):
    """
    :param name: name used to filter inn to Data Elements
    :return:  This method return the basic information about specific data element filtered by id
    """
    try:
        session = login()
        url = f"http://localhost:8080/api/dataElements.json?"
        params = {
            "filter": f"name:like:{name}"
        }
        response = session.get(url, headers={'content-type': 'application/json'}, params=params)
        changeLog(f"[FETCHING DATA-ELEMENT INFO] >> {response.text}")
        return response.text
    except Exception:
        changeLog("Unspected exception")
# Generate a session to makes request
def login():
    session = requests.Session()
    session.auth = ('admin', 'district')
    return session

def createDataElement(code:str,name:str, shortName:str,agregationType:str,domainType:str,valueType:str,zeroIsSignificant:bool):  # create a new data element into dhis
    """
    :param code:
    :param name:
    :param shortName:
    :param agregationType: [SUM,AVERAGE,MIN,MAX,COUNT,FIRST,LAST,STDDEV,VARIANCE,PERCENTILE,MEDIAN]
    :param domainType: [AGGREGATE,TRACKER]
    :param valueType: [TEXT,NUMBER,DATE,BOOLEAN,OPTION (like array list),ORGANISATION_UNIT,FILE_RESOURCE,COORDINATE,URL,CALCULATED_VALUE (for formula)
    :param zeroIsSignificant: True/False
    :return:
    """
    """
    NOTA el category combo es usado para agrupar datos y asociarlos a un id por ejemplo edades o elementos similes
    no es buena idea crear un id combo cada vez que se envian datos, hay que establecer un procedimiento para segun los
    datos que se quieran agrupar se genere un combo nuevo para esos datos
    """
    payload = {
    "code": code,
    "name": name,
    "shortName": shortName,
    "aggregationType": agregationType,
    "domainType": domainType,
    "valueType": valueType,
    "zeroIsSignificant": zeroIsSignificant,
    "categoryCombo": {
        "id": "bjDvmb4bfuf"
        }
    }
    # changeLog("Generated payload with id: "+catComboID)
    # sending data
    try:
        session = login()
        url = "http://localhost:8080/api/dataElements"
        response = session.post(url, json=payload, headers={'content-type': 'application/json'})
        changeLog("[CREATE DATA ELEMENT] >> "+response.text)
    except Exception:
        changeLog("Unspected exception")
def createDataSet(name:str,shortname:str,periodType:str,expiryDays:int,dataElementArr:[],orgUnitArr:[]):
    """
    :param name:
    :param shortname:
    :param periodType: Monthly,
    :param expiryDays: Limits of days until expire the data set ex: 365 or 1000 need to be int data type
    :param dataElementArr: an array list with all ids related to data elements example arrDE = list(["bDH5ZuaeyHH"])
    :param orgUnitArr: an arrat kust with the oute of organizations units starting from the root one
            Example: 'Sierra Leone', 'otherOrgUnit inside'
    :return:
    """

    """
    NOTE
    - The data elements is refered by uid from db and can be seen using '/api/dataElements.json'
    - This method automatically attach the data set to main org unit
    """
    changeLog(f"[CREATE DATA SET] >>  {len(dataElementArr)} DataElement(s) added to DataSet {name}")
    # Generate a data elements list to be added in payload dataSetElements section
    DataElementsList = []
    for x in dataElementArr:
        dataElement = {"dataElement": {"id": x}}
        changeLog(f"[CREATE DATA SET] >>  {x} DataElement added to DataSet {name}")
        DataElementsList.append(dataElement)
    # Generated a OrgUnit element list to be added in payload organisation units section
    OrgUnitList = []
    for x in orgUnitArr:
        OrgUnit = {'id': x}
        changeLog(f"[CREATE DATA SET] >>  {x} OrgUnit added to DataSet {name}")
        OrgUnitList.append(OrgUnit)
    payload = {
        'name': name,  # Name of the dataset
        'shortName': shortname,  # Short name for the dataset
        'periodType': periodType,  # Period type for the dataset
        'expiryDays': expiryDays,  # Number of days until dataset expires
        "dataSetElements": DataElementsList,
        "sharing": {
            "public": "rw------",
            "external": True,
        },
        "organisationUnits": OrgUnitList,
        #ImspTQPwCqd
    }
    try:
        session = login()
        url = "http://localhost:8080/api/dataSets"
        response = session.post(url, json=payload, headers={'content-type': 'application/json'})
        changeLog("[CREATE DATA SET] "+response.text)
    except Exception:
        changeLog("Unspected exception")

# Used to create an organisation unit
def createOrgUnit(code:str,name:str,shortname:str,preriodType:str):
    """
    :param code:
    :param name:
    :param shortname:
    :param preriodType:
    :return:
    """

    """parent its Sierra leone instance or main instance of organization unit"""
    payload = {
        'code': code,
        'name': name,
        'shortName': shortname,
        'openingDate': f"{datetime.date.today().strftime('%Y-%m-%d')}",
        'periodType': preriodType,
        "parent": {
            "id": "ImspTQPwCqd",
        }
    }
    try:
        session = login()
        url = "http://localhost:8080/api/organisationUnits"
        response = session.post(url, json=payload, headers={'content-type': 'application/json'})
        changeLog("[CREATE ORGANISATION UNIT] >> "+response.text)
    except Exception:
        changeLog("Unspected exception")
# Used to upload dava values to specific dara set
def addDataValue(DataValueSets,DataElement:str,DataSetName:str,OrgUnitName:str):
    from datetime import datetime
    """
    :param DataSetName: used to find the id of the dataset
    :param OrgUnitName: used to find the id of the Organization units
                        need the exact structure in dhis for example "Sierra Leone","any other orgunit inside"
    :param DataValueSets: data vec with this format { "dataElement": "dataelement id", "value": "value to upload" }
    :return: none
    """
    # Getting data set id for payload
    DataSetLoaded = json.loads(getDataSetInfoByName(DataSetName))
    DataSetId = DataSetLoaded['dataSets'][0]['id']
    changeLog(f"[FETCHING DATA-SET ID] >> " + DataSetId)

    # Getting orgUnit id for payload
    OrgUnitLoaded = json.loads(getOrgUnitInfoByName(OrgUnitName))
    OrgUnitId = OrgUnitLoaded['organisationUnits'][0]['id']
    changeLog(f"[FETCHING ORG-UNIT ID] >> " + OrgUnitId)

    # Process data value sets for get all ids instead his name
    changeLog(f"[PROCESSING DATA] >> " + "Converting dataelement name to data element id")
    #DataValueSets_loaded = json.loads(DataValueSets)
    dt_info = json.loads(getDataElementInfoByName(DataElement))
    dt_id = dt_info['dataElements'][0]['id']
    # Convertir la cadena a un objeto datetime
    date_obj = datetime.strptime(DataValueSets["date"], "%Y-%m-%d")

    # Convertir el objeto datetime al formato deseado
    formatted_date = date_obj.strftime("%YW%U")
    payload = {
        "dataSet": DataSetId,
        "completeDate": DataValueSets["date"],
        "period": formatted_date,
        "orgUnit": OrgUnitId,
        "dataValues": [{"dataElement":dt_id,"value":DataValueSets["Value"]}]
    }
    try:
        session = login()
        url = "http://localhost:8080/api/dataValueSets"
        response = session.post(url, json=payload, headers={'content-type': 'application/json'})
        changeLog("[INSERT DATA-VALUE-SET] >> "+response.text)
    except Exception:
        changeLog("Unspected exception")


In [16]:
def getDataValueSetByDataSetID(datasetID:str):
    try:
        session = login()
        url = f"http://localhost:8080/api/dataSets/{datasetID}/dataValueSet"
        response = session.get(url)
        changeLog("[FETCHING DATA-VALUE-SET INFO] >> "+ response.text)
        return response.text
    except Exception:
        changeLog("Unspected exception")
        

        

In [17]:
def getCategoryOptionCombo(datasetID:str,dataElementID:str):
    DataValueSetLoaded = json.loads(getDataValueSetByDataSetID(datasetID))
    for i in DataValueSetLoaded['dataValues']:
        if i["dataElement"] == dataElementID:
            return i["categoryOptionCombo"]
        else: 
            changeLog(f"Cant find categoryOptionCombo for data element {dataElementID} in dataset {datasetID}")

In [18]:
def addDataValue_one_Value_test(value:int, date):
    import json
    import requests
    import pandas as pd
    from datetime import datetime
    OrgUnitName = "Canarias"
    DataElement = "IDS - invented (Deaths) "
    DataSetName = 'new_dataset'
    DataElement = 'Yellow Fever (Suspected cases)'
    DataSetName = 'IDS - Report: Suspected, Confirm, Death'
    """
    :param DataSetName: used to find the id of the dataset
    :param OrgUnitName: used to find the id of the Organization units
                        need the exact structure in dhis for example "Sierra Leone","any other orgunit inside"
    :param DataValueSets: data vec with this format { "dataElement": "dataelement id", "value": "value to upload" }
    :return: none
    """
    # Getting data set id for payload
    DataSetLoaded = json.loads(getDataSetInfoByName(DataSetName))
    DataSetId = DataSetLoaded['dataSets'][0]['id']
    changeLog(f"[FETCHING DATA-SET ID] >> " + DataSetId)

    OrgUnitLoaded = json.loads(getOrgUnitInfoByName(OrgUnitName))
    OrgUnitId = OrgUnitLoaded['organisationUnits'][0]['id']
    changeLog(f"[FETCHING ORG-UNIT ID] >> " + OrgUnitId)

    # Process data value sets for get all ids instead his name
    changeLog(f"[PROCESSING DATA] >> " + "Converting dataelement name to data element id")
    #DataValueSets_loaded = json.loads(DataValueSets)
    dt_info = json.loads(getDataElementInfoByName(DataElement))
    dt_id = dt_info['dataElements'][0]['id']
    # Convertir la cadena a un objeto datetime
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    print(DataSetId)
    categoryOptionComboID = getCategoryOptionCombo(DataSetId, dt_id)
  # Convertir el objeto datetime al formato deseado
    formatted_date = date_obj.strftime("%YW%U")
    payload = {
    "dataSet": DataSetId,
    "completeDate": date,
    "orgUnit": OrgUnitId,
    "dataValues": 
        [{'dataElement': dt_id,
       'categoryOptionCombo': categoryOptionComboID,
       'period': formatted_date,
       'value': value,
        'followup': True}]
    }
    try:
        session = login()
        url = "http://localhost:8080/api/dataValueSets"
        response = session.post(url, json=payload, headers={'content-type': 'application/json'})
        changeLog("[INSERT DATA-VALUE-SET] >> "+response.text)
        return response
    except Exception:
        changeLog("Unspected exception")

In [19]:
# response = addDataValue_one_Value_test(10,"2023-07-16")

In [20]:
import datetime

def generate_day_a_week(year,week_day_number):
    start_date = datetime.date(year, 1, 1)  # primer día del año
    end_date = datetime.date(year, 12, 31)  # último día del año
    delta = datetime.timedelta(days=1)

    week_day = []
    current_date = start_date

    while current_date <= end_date:
        if current_date.weekday() == week_day_number:  # 0 es lunes
            week_day.append(current_date.strftime('%Y-%m-%d'))
        current_date += delta

    return week_day

In [21]:
def export_random_data_test(orgUnit:str,years:list):
    import pandas as pd
    dataElement = 'Yellow Fever (Suspected cases)'
    dataSet = 'IDS - Report: Suspected, Confirm, Death'
    orgUnit = orgUnit  
    file = "extrapolated_data.csv"
    data = pd.read_csv(file)
    # Convertir la columna 'Year' a formato de fecha y hora
    data['Year'] = pd.to_datetime(data['Year'])
    week_days = list()
    for y in years:
        week_days.extend(generate_day_a_week(y,1))
    data = data[:len(week_days)]
    data["date"] = week_days
    data.apply(addDataValue, axis = 1, args=(dataElement,dataSet,orgUnit,))
    return data
    
    

In [22]:
def impoer_payload(payload):
    import pandas as pd
    dataElement = 'Yellow Fever (Suspected cases)'
    dataSet = 'IDS - Report: Suspected, Confirm, Death'
    orgUnit = orgUnit  
    file = "extrapolated_data.csv"
    data = pd.read_csv(file)
    # Convertir la columna 'Year' a formato de fecha y hora
    data['Year'] = pd.to_datetime(data['Year'])
    week_days = list()
    for y in years:
        week_days.extend(generate_day_a_week(y,1))
    data = data[:len(week_days)]
    data["date"] = week_days
    data.apply(addDataValue, axis = 1, args=(dataElement,dataSet,orgUnit,))
    return data

In [23]:
def  get_dataValues(dataSet, orgUnit):
    datasetID = json.loads(getDataSetInfoByName(dataSet))["dataSets"][0]["id"]
    orgUnitID = json.loads(getOrgUnitInfoByName(orgUnit))["organisationUnits"][0]["id"]

    try:
        session = login()
        url = f"http://localhost:8080/api/dataValueSets.json?dataSet={datasetID}&period=2023W1&orgUnit={orgUnitID}"
#         params = {
#             "filter": f"dataSet:like:{datasetID}&orgUnit:like:{orgUnitID}"
#         }
        response = session.get(url) 
                               #headers={'content-type': 'application/json'}, params=params)
        changeLog(f"[FETCHING DATA-ELEMENT INFO] >> {response.text}")
        return response.text
    except Exception:
        changeLog("Unspected exception")

In [27]:
export_random_data_test("Trarza", [2023])

,Year,Value,date
0,2007-01-01,905,2023-01-03
1,2007-01-01,2442,2023-01-10
2,2007-01-01,2223,2023-01-17
3,2007-01-01,2839,2023-01-24
4,2007-01-01,2736,2023-01-31
5,2007-01-01,2565,2023-02-07
6,2007-01-01,2061,2023-02-14
7,2007-01-01,2344,2023-02-21
8,2007-01-01,1959,2023-02-28
9,2007-01-01,1925,2023-03-07


2023-07-19 13:18:34.942827: [FETCHING DATA-ELEMENT INFO] >> {"pager":{"page":1,"total":1,"pageSize":50,"pageCount":1},"dataElements":[{"displayName":"IDS - Yellow Fever (Suspected cases)","id":"Q1xDDDusLwl"}]}

2023-07-19 13:18:35.067180: [INSERT DATA-VALUE-SET] >> {"httpStatus":"Conflict","httpStatusCode":409,"status":"WARNING","message":"One more conflicts encountered, please check import summary.","response":{"responseType":"ImportSummary","status":"WARNING","importOptions":{"idSchemes":{},"dryRun":false,"async":false,"importStrategy":"CREATE_AND_UPDATE","mergeMode":"REPLACE","reportMode":"FULL","skipExistingCheck":false,"sharing":false,"skipNotifications":false,"skipAudit":false,"datasetAllowsPeriods":false,"strictPeriods":false,"strictDataElements":false,"strictCategoryOptionCombos":false,"strictAttributeOptionCombos":false,"strictOrganisationUnits":false,"strictDataSetApproval":false,"strictDataSetLocking":false,"strictDataSetInputPeriods":false,"requireCategoryOptionCombo":false,"requireAttributeOptionCombo":false,"skipPatternValidation":false,"ignoreEmptyCollection":false,"force":false,"firstRowIsHeader":true,"skipLastUpdated":false,"mergeDataValues":false,"skipCache":false},"description":"Import process completed successfully","importCount":{"imported":0,"updated":0,"ignored":1,"deleted":0},"conflicts":[{"objects":{"dataElement":"Q1xDDDusLwl","dataSet":"ZyZmZTUwctj"},"value":"Period: `null` is after latest open future period: `2023W29` for data element: `Q1xDDDusLwl` and data set: `ZyZmZTUwctj`","errorCode":"E7641","property":"period","indexes":[0]}],"dataSetComplete":"2023-12-26"}}


In [4]:
def generar_numero_ponderado(n, media):
    numeros = list(range(1, n+1))
    probabilidades = [1 / abs(x - media) if x != media else 1 for x in numeros]
    total_probabilidades = sum(probabilidades)
    ponderaciones = [p / total_probabilidades for p in probabilidades]

    numero_aleatorio = random.choices(numeros, weights=ponderaciones)[0]
    return numero_aleatorio


In [10]:
l = [generar_numero_ponderado(10,1) for n in range(10)]